In [1]:
import os
import re
import sys
import numpy as np

In [2]:
def read_agecker_Macaque_wild(folder_path, scale_size = (1920, 1080)):
    width, height = scale_size
    data_names = os.listdir(folder_path)
    data_names = [name for name in data_names if re.search(".txt",name)]
    frame_nr = []
    class_name = []
    id_nr = []
    bbox_xywh = []
    for data_name in data_names:
        maca_data = open(os.path.join(folder_path,data_name)).read().strip().split("\n")
        if maca_data == [""]:
            continue
        maca_data = [[float(nr) for nr in row.strip().split(" ")] for row in maca_data]
        frame_name = data_name.split(".")[0]
        #-1 that mot also starts at 0
        frame_nr_to_append = int(re.findall("[0-9]+",frame_name)[0])
        for row in maca_data:
            frame_nr.append(frame_nr_to_append)
            class_name.append(row[0])
            id_nr.append(row[1])
            #to upper left corner from middle 
            row[2] = (float(row[2]) - float(row[4])/2) * width
            row[3] = (float(row[3]) - float(row[5])/2) * height
            row[4] = float(row[4]) * width
            row[5] = float(row[5]) * height
            row[2:6] = [round(x,2) for x in row[2:6]]
            bbox_xywh.append(row[2:6])
    #now bring them in the right order of the frames for sure:
    order_to_follow = np.argsort(frame_nr)
    frame_nr = [frame_nr[i] for i in order_to_follow]
    id_nr = [id_nr[i] for i in order_to_follow]
    bbox_xywh = [bbox_xywh[i] for i in order_to_follow]
    class_name = [class_name[i] for i in order_to_follow]
    return {"frame_name":frame_nr,"id_nr":id_nr,"bbox_xywh":bbox_xywh,"class_name":class_name}

def write_MOT(path, dict_to_write):
    """
    Write the MOT file for the given dictionary

    path: path to write the file
    mot_dict: dictionary with MOT data
    """
    frame_nr = dict_to_write["frame_name"]
    id_nr = dict_to_write["id_nr"]
    bbox_xywh = dict_to_write["bbox_xywh"]
    confidence_score = dict_to_write["confidence_score"]
    with open(path, "w") as f:
        for i in range(len(frame_nr)):
            row = [str(frame_nr[i]+1), str(int(id_nr[i])), *[str(round(x, 2)) for x in bbox_xywh[i]], str(round(confidence_score[i], 2)),str(1),str(1)]
            f.write(",".join(row) + "\n")



In [3]:
base_path = "/usr/users/vogg/macaque_videos_eval/"
videos = os.listdir(base_path)
videos = [vid for vid in videos if re.search("VID", vid)]
for video in videos:
    seqLength = len(os.listdir(os.path.join(base_path, video, "images")))
    with open(os.path.join(base_path, video, "seqinfo.ini"), "w") as to_write:
        to_write.write("[Sequence]\n")
        to_write.write("; name=MOT16-01\n")
        to_write.write(f"; imDir={video}/images\n")
        to_write.write("frameRate=30\n")
        to_write.write(f"seqLength={seqLength}\n")
        to_write.write("imWidth=1920\n")
        to_write.write("imHeight=1080\n")
        to_write.write("imExt=.jpg\n")

for video in videos:
    infos = read_agecker_Macaque_wild(os.path.join(base_path, video, "labels_with_ids"), scale_size = (1920, 1080))
    print(video)
    infos["confidence_score"] = [1 for _ in infos["id_nr"]]
    write_MOT(os.path.join(base_path, video, "gt.txt"), infos)



VID_20210301_105722_0
VID_20210223_123630_0
VID_20210227_133440_0
VID_20210228_154053_0
VID_20210302_103130_0
VID_20210301_151229_0
VID_20210228_160721_0
VID_20210224_114038_0
VID_20210224_115729_0
VID_20210223_123817_0
VID_20210301_145312_0
VID_20210228_153846_0
VID_20210224_115455_0
VID_20210223_123854_0
VID_20210228_153942_0
VID_20210301_143635_0
VID_20210302_103307_0
VID_20210227_133251_0
